In [2]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openai]2m1/2 [openai]


In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="nvidia/llama-3.3-nemotron-super-49b-v1.5",
  messages=[{"role":"system","content":"/think"}],
  temperature=0.6,
  top_p=0.95,
  max_tokens=65536,
  frequency_penalty=0,
  presence_penalty=0,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



In [ ]:
import os
import json
import re
import pandas as pd
from openai import OpenAI
from tqdm import tqdm

# =========================
# 설정
# =========================
INPUT_FILE_PATH = "/home/jovyan/work/GitHub/AIFFEL_quest_rs/GoingDeeper/Deeper05/ChatbotData.csv"
OUTPUT_FILE_PATH = "augmented_data.csv"

COL_QUESTION = "Q"
COL_ANSWER   = "A"

client = OpenAI(
    api_key="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",   # export NVIDIA_API_KEY="..."
    base_url="https://integrate.api.nvidia.com/v1"
)

MODEL_ID = "nvidia/llama-3.3-nemotron-super-49b-v1.5"


import re, json
import pandas as pd
from tqdm import tqdm

def generate_augmented_data_stream(question, answer, num_pairs=2):
    prompt = f"""
너는 챗봇 학습 데이터 증강 전문가다.
아래 원본 Q/A로부터 {num_pairs}개의 새로운 Q/A를 생성하라.

[필수 규칙]
1) 출력은 JSON만: {{"pairs":[{{"question":"...","answer":"..."}}, ...]}}
2) question은 원문을 그대로 복사하면 안 된다. (완전 동일/정규화 동일 모두 금지)
3) question은 원문상황에서 이어서 나올 수 있는 질문으로 바꿔라: 구어체/줄임말 다른 말로 표현 등을 섞어라.
4) {num_pairs}개의 question은 서로도 겹치면 안 된다.
5) answer도 원문을 그대로 복사하면 안 된다.(완전 동일/정규화 동일 모두 금지)
Q: {question}
A: {answer}
""".strip()

    schema = {
        "name": "qa_augmentation",
        "strict": True,
        "schema": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "pairs": {
                    "type": "array",
                    "minItems": num_pairs,
                    "items": {
                        "type": "object",
                        "additionalProperties": False,
                        "properties": {
                            "question": {"type": "string"},
                            "answer": {"type": "string"}
                        },
                        "required": ["question", "answer"]
                    }
                }
            },
            "required": ["pairs"]
        }
    }

    completion = client.chat.completions.create(
        model="nvidia/llama-3.3-nemotron-super-49b-v1.5",
        messages=[
            {"role": "system", "content": "Return ONLY JSON that matches the schema."},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_schema", "json_schema": schema},
        temperature=0.7,
        top_p=0.95,
        max_tokens=1200,
        stream=True
    )

    full_text = ""
    for chunk in completion:
        delta = chunk.choices[0].delta
        if delta and delta.content:
            full_text += delta.content

    data = json.loads(full_text)
    return data["pairs"]

def main():
    try:
        df = pd.read_csv(INPUT_FILE_PATH, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(INPUT_FILE_PATH, encoding="cp949")

    if COL_QUESTION not in df.columns or COL_ANSWER not in df.columns:
        raise ValueError(f"CSV에 '{COL_QUESTION}', '{COL_ANSWER}' 컬럼이 없습니다. 현재: {list(df.columns)}")

    augmented_rows = []
    error_count = 0
    empty_pairs_count = 0

    for index, row in tqdm(df.iterrows(), total=len(df)):
        q_text = row[COL_QUESTION]
        a_text = row[COL_ANSWER]

        if pd.isna(q_text) or pd.isna(a_text):
            continue

        q_text = str(q_text)
        a_text = str(a_text)

        augmented_rows.append({COL_QUESTION: q_text, COL_ANSWER: a_text, "type": "original"})

        try:
            pairs = generate_augmented_data_stream(q_text, a_text, num_pairs=3)
        except Exception as e:
            error_count += 1
            pairs = []
            print(f"\n[ERROR] index={index} {e}")

        if not pairs:
            empty_pairs_count += 1

        for pair in pairs:
            new_q = pair.get("question") or pair.get("Question") or pair.get("Q")
            new_a = pair.get("answer")   or pair.get("Answer")   or pair.get("A")
            if new_q and new_a:
                augmented_rows.append({COL_QUESTION: new_q, COL_ANSWER: new_a, "type": "augmented"})

        if (index + 1) % 50 == 0:
            aug_count = sum(1 for r in augmented_rows if r["type"] == "augmented")
            print("\n" + "=" * 90)
            print(f"[{index+1}/{len(df)}] total_rows={len(augmented_rows)}  augmented={aug_count}  empty_pairs={empty_pairs_count}  errors={error_count}")
            print("ORIGINAL Q:", q_text)
            print("ORIGINAL A:", a_text[:200], "...")
            if pairs:
                print("AUG Q:", (pairs[0].get("question") or "")[:200])
                print("AUG A:", (pairs[0].get("answer") or "")[:200], "...")
            else:
                print("(no augmented pairs)")

        if (index + 1) % 200 == 0:
            pd.DataFrame(augmented_rows).to_csv("aug_checkpoint.csv", index=False, encoding="utf-8-sig")
            print(f"\n[checkpoint saved] aug_checkpoint.csv rows={len(augmented_rows)}")

    out_df = pd.DataFrame(augmented_rows)
    out_df.to_csv(OUTPUT_FILE_PATH, index=False, encoding="utf-8-sig")
    print(f"\n✅ 저장 완료: {OUTPUT_FILE_PATH} (총 {len(out_df)}행)")
    print(f"📌 summary: empty_pairs={empty_pairs_count}, errors={error_count}")

if __name__ == "__main__":
    main()



  0%|          | 50/11823 [01:39<5:17:03,  1.62s/it]


[50/11823] total_rows=200  augmented=150  empty_pairs=0  errors=0
ORIGINAL Q: 감 말랭이 먹고 싶다.
ORIGINAL A: 맛있게 드세요. ...
AUG Q: 감 말랭이 먹고 싶은데 어때?
AUG A: 맛있겠네요, 즐거운 식사되세요! ...


  1%|          | 100/11823 [03:20<7:47:00,  2.39s/it]


[100/11823] total_rows=400  augmented=300  empty_pairs=0  errors=0
ORIGINAL Q: 거지 같이 일해 놓고 갔어
ORIGINAL A: 일 못하는 사람이 있으면 옆에 있는 사람이 더 힘들죠. ...
AUG Q: 이번에 작업을 하다가 동료가 너무 일도 못하고 떠났어. 어떻게 해야 할까?
AUG A: 동료가 일을 맡고 떠났을 때는 주변 사람들과 협력하여 업무를 나누는 것이 중요합니다. 또한, 향후에는 업무 분배 시 신중하게 계획하는 것이 좋습니다. ...


  1%|▏         | 150/11823 [05:02<6:25:57,  1.98s/it]

...

 36%|███▋      | 4300/11823 [2:21:44<4:32:53,  2.18s/it]


[4300/11823] total_rows=17199  augmented=12899  empty_pairs=1  errors=1
ORIGINAL Q: 지하철에서 괜찮은 사람 봤는데 말 시켜 볼까
ORIGINAL A: 용기가 필요한 순간이네요. ...
AUG Q: 지하철에서 좋은 사람 만났는데 접근해볼까요?
AUG A: 용기를 내는 것도 중요하죠. ...


 37%|███▋      | 4350/11823 [2:23:31<3:50:45,  1.85s/it]


[4350/11823] total_rows=17399  augmented=13049  empty_pairs=1  errors=1
ORIGINAL Q: 집나가도 갈 데가 없어
ORIGINAL A: 선생님이나 기관에 연락해보세요. ...
AUG Q: 집을 나와야 하지만 어디로 가야 할지 모르겠어요. 도움을 받을 수 있는 방법이 있을까요?
AUG A: 학교 선생님이나 유소년 축구 기관에 연락하여 도움을 요청하세요. ...


 37%|███▋      | 4392/11823 [2:25:02<4:17:18,  2.08s/it]